# Workflow test

**TODO:**

- Explore the sqlite output db
- Look through the NBs and see how they visualize the output
  - Where are the identified XLs filtered based on the MS2 data...?
    - Look through `fig_maker.py`, `DDA_clean.py`, `fragment_generator.py`, and `fragments.py`
- Make a dict that holds file information (like model number)

# "Global" imports and variable assignments

In [1]:
from pathlib import Path
from glob import glob

out_dir_base = Path(
    '/home/jo0348st/resources/projects/2023-heusel_m1_abs/results')

out_dir = out_dir_base / '230105/txms_m1-c_rep'

cut_off = 30  # Distance cutoff (Å)

# 7. Modeling (without the modeling)

**TODO:**

- Option to input docked complex (one file) or un-docked structures (two files)
- Option to set the atoms to calculate the euclidean distance between (e.g. alpha/beta/omega carbon, or nitrogen)

In [ ]:
# Imports and variable assignments
from utils import rosettaxlv, rosettaxl

# Import and initiate Rosetta
import pyrosetta
import pyrosetta.rosetta as rosetta
from pyrosetta import init, Pose, pose_from_pdb, PyJobDistributor, create_score_function
from rosetta.protocols.rigid import *

init()


# Set input variables (according to vars.ini)
input_pdb = './data/uhD4YUPj6ImEQJi/pdb_C.pdb'
partners = 'A_B'  # Binding partners (chain labels)
top_XL_file = './data/uhD4YUPj6ImEQJi/top_XL.txt'  # Reproduce this
num_of_top_filters = 2

# Grab model paths
input_models = glob('./data/uhD4YUPj6ImEQJi/LR_models/*.pdb')

In [ ]:
# Score models based on XLs

score_t_list = []
score_normal_list = []
pose_list = []
out_all_XLs_list = []
out_XLs_list = []
XL_below_cutoff_list = []


if len(input_models) > 0:

    for model_path in input_models:

        print(model_path)

        dock_pose = pose_from_pdb(model_path)

        score_t, score_normal, XL_below_cutoff = rosettaxlv.rosettaxlv(

            dock_pose,
            top_XL_file,
            cut_off

        )

        if len(score_t_list) > num_of_top_filters:
            min_score = min(score_t_list)
            index_min = score_t_list.index(min_score)
            if score_t > min_score:
                score_t_list[index_min] = score_t
                score_normal_list[index_min] = score_normal
                pose_list[index_min] = dock_pose

                XL_below_cutoff_list[index_min] = XL_below_cutoff

        else:
            score_t_list.append(score_t)
            score_normal_list.append(score_normal)
            pose_list.append(dock_pose)

            XL_below_cutoff_list.append(XL_below_cutoff)

    for num_pos, struct in enumerate(pose_list):

        pdb_name = Path(f'lr_model_{str(num_pos)}')

        full_path = str(out_dir / pdb_name)

        files = [

            x for y in ('*.fasc', '*.pdb', '*.txt')

            for x in out_dir.glob(f'{pdb_name}{y}')

        ]

        if len(files) > 0:

            for f in files:

                Path(f).unlink()

        out_all_name = out_dir.joinpath(f'{pdb_name}_all_XLs.txt')

        out_all_XLs_list.append(str(out_all_name))

        scorefxn_low = create_score_function('interchain_cen')

        jd = PyJobDistributor(full_path, 1, scorefxn_low)

        struct.pdb_info().name(full_path + '_fa')

        jd.output_decoy(struct)

        rosettaxl.rosettaxl(struct, partners, cut_off, str(out_all_name))

        out_name = out_dir.joinpath(f'{pdb_name}_XLs.txt')

        out_XLs_list.append(out_name)

        with open(out_name, 'w') as f:

            XLs = XL_below_cutoff_list[num_pos]

            for XL in XLs:

                print(XL, file=f)

In [ ]:
# CHECK OUT THE LISTS AND WRITE A CSV OR SQLITE DB

## Potential issue

Difference in the output XLs between the nR and R workflows. Where the euclidean distance seems to be calculated in the same way (_needs further investigation_)—on the same atom (CB)—but reports different distances and therefore filters the peptides differently.

# 8. Convert mzML to MGF

Write wrapper for `msconvert`, using `subprocess.run()`.

In [ ]:
#import subprocess

# 9. Search MS2 for XLs supported by the predicted structures

**TODO:**

- _Try to find where the actual cutoff is used_
- **Speed up the script** by cleaning the MGF before running TaxLink, using a concatenated file with all XLs
    - Make sure that this makes sense with the script used to filter the MGF file

In [ ]:
# Imports and variable assignments.
from utils import TaxLink

xl_files = [str(x) for x in out_XLs_list]
mgf_file = './data/uhD4YUPj6ImEQJi/ms2.mgf'
delta = 0.01
intensity = 0.0

score = 0
score_list = []

In [ ]:
for xl_file in xl_files:

    # Model number assignment that works with current hardcoding...
    # Could alternatively be "hardcoded" by enumerating the file list.
    model_nr = xl_file.rstrip('_XLs.txt')[-1]

    score = TaxLink.taxlink(

        xl_file,
        mgf_file,
        delta,
        intensity,
        out_dir,
        model_nr

)

    score_list.append(score)

top_model_xl = xl_files[score_list.index(max(score_list))]

top_model = f'{top_model_xl[:-7]}0.pdb'

print('TOP MODEL: ', top_model)

command_1 = ['cp', top_model, f'results/test_out/best_model.pdb']
subprocess.run(command_1)

command_2 = ['cp', top_model_xl, f'results/test_out/best_model_XLs.txt']
subprocess.run(command_2)

In [ ]:
# Check the SQLite DBs
# import pandas as pd
# import numpy as np
# import sqlite3 as sql


# db = './data/uhD4YUPj6ImEQJi/xldist_db'
# con = sql.connect(db)

# query = """SELECT * FROM xldist
#     WHERE dist<30.0
#     AND pdb_filename NOT LIKE '%best_model%'"""

# df = pd.read_sql_query(query, con)

# df

# 10. Create pyMOL session

In [15]:
import sqlite3 as sql
import __main__
__main__.pymol_argv = ['pymol', '-qc']

import pymol
from pymol import cmd
from pyrosetta import init, pose_from_pdb, Pose, pose_from_file

from notebook_utils import rosettaxlv, rosettaxl

init()

pdb_file = out_dir / 'best_model.pdb'
top_xls_file = out_dir / 'best_model_xl.txt'
ms2_db = out_dir / 'ms2_results.sql'

PyRosetta-4 2022 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2022.41+release.28dc2a1757fe7c8bf9b661dd8d6cd903c305e581 2022-10-14T12:31:19] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r333 2022.41+release.28dc2a1 28dc2a1757fe7c8bf9b661dd8d6cd903c305e581 http://www.pyrosetta.org 2022-10-14T12:31:19
core.init: command: PyRosetta -ex1 -ex2aro -database /srv/data1/home/jo0348st/envs/miniconda3/envs/pyrosetta/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1561373534 seed_offset=0 real_seed=-1561373534 thread_index=0
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1561373534 RG_t

In [3]:
con = sql.connect(ms2_db)
con_c = con.cursor()
query = f"""SELECT * FROM MS2Data"""  # top_model_xl
con_c.execute(query)
table = con_c.fetchall()

ms2_xl_list = []
for row in table:
    ms2_xl_list.append(str(row[0]))

con.close()

In [17]:
pose = pose_from_file(str(pdb_file))
sequence = pose.sequence()

pymol.finish_launching()
p_name = pdb_file.stem
cmd.bg_color("white")
cmd.load(pdb_file)
cmd.set_title(p_name, 1, '')
cmd.show_as("cartoon", p_name)
cmd.color("gray", p_name + " and name CA")

core.import_pose.import_pose: File '/home/jo0348st/resources/projects/2023-heusel_m1_abs/results/230105/txms_m1-c_rep/best_model.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] skipping pdb residue b/c it's missing too many mainchain atoms:  121 A ALA ALA
core.io.pose_from_sfr.PoseFromSFRBuilder: missing:  N
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] skipping pdb residue b/c it's missing too many mainchain atoms:  445 B GLU GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: missing:  N
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 2 atoms at position 1 in file /home/jo0348st/resources/projects/2023-heusel_m1_abs/results/230105/txms_m1-c_rep/best_model.pdb. Best match rsd_type:  ALA:NtermProteinFull
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 1 atoms at position 120 in file /home/jo0348st/resources/projects/2023-heusel_m1_abs/results/230105/txms_m1-c_rep/best_model.pdb. Best match rsd_typ

1

In [ ]:
from string import digits

output_xl_number = 0
found_XL_list = []

for num_xl, xl in enumerate(ms2_xl_list):
    #print(num_xl+1, xl)

    K_pos_P1 = 0
    K_pos_P2 = 0
    eulidean_dist = 10000.0

    xl_trans = str.maketrans('', '', digits)
    xl_without_digit = xl.translate(xl_trans)
    peptide1 = xl_without_digit.split(
            '--')[0].replace("-", "").replace(".", "")[:-2]
    peptide2 = xl_without_digit.split('--')[1].replace("-", "")[:-3]

    K_pos_P1 = peptide1.find('K') + 1
    K_pos_P2 = peptide2.find('K') + 1

    multiple_occ_list1 = [x for x in range(
        len(sequence)) if sequence.find(peptide1, x) == x]
    multiple_occ_list2 = [y for y in range(
        len(sequence)) if sequence.find(peptide2, y) == y]

    ## finding minimum distance if multiple occurance happened
    seq_pos_p1_k = 0
    seq_pos_p2_k = 0
    tmp_dist = 10000.0
    for pos1 in multiple_occ_list1:
        for pos2 in multiple_occ_list2:
            if tmp_dist > rosettaxl.rosetta_eu_dist(pose, pos1+K_pos_P1, pos2+K_pos_P2):
                tmp_dist = rosettaxl.rosetta_eu_dist(
                    pose, pos1+K_pos_P1, pos2+K_pos_P2)
                seq_pos_p1_k = pos1+K_pos_P1
                seq_pos_p2_k = pos2+K_pos_P2

    #print("aa positions on the sequence: ", seq_pos_p1_k, seq_pos_p2_k)

    if ((peptide1 in sequence) and (peptide2 in sequence)):
        eulidean_dist = rosettaxl.rosetta_eu_dist(
            pose, seq_pos_p1_k, seq_pos_p2_k)

        #print("Euclidean distance is:  ", eulidean_dist, "\n")

    else:
        print("The XL is not found on the protein sequence. Check each peptide to be valid!")
        print('Seq:', sequence)
        print('XL:', peptide1, '--', peptide2)

    if eulidean_dist <= cut_off:
        output_xl_number += 1
        found_XL_list.append(xl)
        cmd.distance("dist_"+str(num_xl+1),
                        str(seq_pos_p1_k)+"/CA", str(seq_pos_p2_k)+"/CA")

## writing XLs below threshold in a file
# for item in found_XL_list:
#     found_xl_file.write("%s\n" % item)


cmd.set('dash_color', 'red')
cmd.set('dash_width', 4)
cmd.set('label_size', 22)
pymol.cmd.save(out_dir / "pymol_result.pse")

# Get out!
pymol.cmd.quit()

(!!!) **Figure out how to assign biological units in a PDB file**